In [1]:
!pip install tensorflow

  Using cached tensorflow-2.20.0-cp313-cp313-win_amd64.whl.metadata (4.6 kB)
  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached termcolor-3.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached grpcio-1.74.0-cp313-cp313-win_amd64.whl.metadata (4.0 kB)
  Using cached tensorboard-2.20.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached keras-3.11.3-py3-none-any.whl.metadata (5.9 kB)
  Using cached ml_dtypes-0.5.3-cp313-cp313-win_amd64.whl.metadata (9.2 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
  Using c

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
import zipfile
import os

# 1. Provide the full, correct path to the zip file on your PC.
#    Using forward slashes '/' is the easiest way to avoid errors in Python.
zip_path = 'C:/Users/Hp/Downloads/banana_classification.zip'

# 2. Define a simple name for the folder where files will be extracted.
#    This folder will be created in the same directory as your notebook.
extract_path = 'banana_dataset'

# --- No need to change anything below this line ---

# Check if the zip file actually exists before trying to open it
if not os.path.exists(zip_path):
    print(f"ERROR: The file was not found at '{zip_path}'")
    print("Please double-check the path and filename.")
else:
    # Create the extraction directory if it doesn't exist
    os.makedirs(extract_path, exist_ok=True)

    # Unzip the file
    print(f"Unzipping '{zip_path}'...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

    print(f"✅ Files extracted successfully to the '{extract_path}' folder!")

Unzipping 'C:/Users/Hp/Downloads/banana_classification.zip'...
✅ Files extracted successfully to the 'banana_dataset' folder!


In [26]:
train_dir = "banana_dataset/banana_classification/train"
valid_dir = "banana_dataset/banana_classification/valid"
test_dir = "banana_dataset/banana_classification/test"

In [27]:
img_size = (128, 128)
batch = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True
)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen  = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    train_dir, target_size=img_size, batch_size=batch, class_mode='categorical')
valid_gen = valid_datagen.flow_from_directory(
    valid_dir, target_size=img_size, batch_size=batch, class_mode='categorical')
test_gen = test_datagen.flow_from_directory(
    test_dir, target_size=img_size, batch_size=batch, class_mode='categorical', shuffle=False)


Found 11793 images belonging to 4 classes.
Found 1123 images belonging to 4 classes.
Found 562 images belonging to 4 classes.


In [29]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(*img_size, 3)),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(4, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


C:\Users\Hp\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,305,156 (12.61 MB)

 Trainable params: 3,305,156 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [30]:
history = model.fit(
    train_gen,
    validation_data=valid_gen,
    epochs=10   # increase if needed
)


C:\Users\Hp\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
369/369 ━━━━━━━━━━━━━━━━━━━━ 323s 867ms/step - accuracy: 0.6828 - loss: 0.7303 - val_accuracy: 0.8451 - val_loss: 0.3837
Epoch 2/10
369/369 ━━━━━━━━━━━━━━━━━━━━ 180s 487ms/step - accuracy: 0.8543 - loss: 0.3925 - val_accuracy: 0.9145 - val_loss: 0.2648
Epoch 3/10
369/369 ━━━━━━━━━━━━━━━━━━━━ 178s 482ms/step - accuracy: 0.8878 - loss: 0.3094 - val_accuracy: 0.9234 - val_loss: 0.2182
Epoch 4/10
369/369 ━━━━━━━━━━━━━━━━━━━━ 180s 487ms/step - accuracy: 0.9067 - loss: 0.2665 - val_accuracy: 0.9181 - val_loss: 0.2398
Epoch 5/10
369/369 ━━━━━━━━━━━━━━━━━━━━ 240s 650ms/step - accuracy: 0.9132 - loss: 0.2485 - val_accuracy: 0.9412 - val_loss: 0.1863
Epoch 6/10
369/369 ━━━━━━━━━━━━━━━━━━━━ 231s 623ms/step - accuracy: 0.9204 - loss: 0.2250 - val_accuracy: 0.9305 - val_loss: 0.1915
Epoch 7/10
369/369 ━━━━━━━━━━━━━━━━━━━━ 220s 596ms/step - accuracy: 0.9239 - loss: 0.2147 - val_accuracy: 0.9537 - val_loss: 0.1644
Epoch 8/10
369/369 ━━━━━━━━━━━━━━━━━━━━ 220s 595ms/step - accuracy: 0.9290 -

In [31]:
loss, acc = model.evaluate(test_gen)
print(f"Test Accuracy: {acc:.2f}")


C:\Users\Hp\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


18/18 ━━━━━━━━━━━━━━━━━━━━ 8s 431ms/step - accuracy: 0.9306 - loss: 0.1673
Test Accuracy: 0.93


In [ ]:
from tensorflow.keras.preprocessing import image

class_to_days = {
    "unripe": "≈ 5-6 days left",
    "ripe": "≈ 2-3 days left",
    "overripe": "≈ 1 day left",
    "rotten": "0 days (already dead)"
}

# save mapping from generator
labels = list(train_gen.class_indices.keys())

def predict_days(img_path):
    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    preds = model.predict(img_array)
    class_idx = np.argmax(preds[0])
    predicted_class = labels[class_idx]
    return predicted_class, class_to_days[predicted_class]

# Example usage:
test_folder = 'banana_dataset/banana_classification/test/rotten/'
test_images = os.listdir(test_folder)
for i in range(len(test_images)):
    banana_stage, days_left = predict_days(os.path.join(test_folder, test_images[i]))
    print(f"Stage: {banana_stage}, Days left: {days_left}")
    print(i)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step
Stage: rotten, Days left: 0 days (already dead)
0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Stage: rotten, Days left: 0 days (already dead)
1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
Stage: rotten, Days left: 0 days (already dead)
2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
Stage: rotten, Days left: 0 days (already dead)
3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
Stage: rotten, Days left: 0 days (already dead)
4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Stage: rotten, Days left: 0 days (already dead)
5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Stage: rotten, Days left: 0 days (already dead)
6
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Stage: rotten, Days left: 0 days (already dead)
7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
Stage: rotten, Days left: 0 days (already dead)
8
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Stage: rotten, Days left: 0 days (already dead)
9
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Stage: rotten, Days left: 0 days (already dead)
10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42

In [52]:
# This will print the class-to-index mapping the model learned
print(train_gen.class_indices)

{'overripe': 0, 'ripe': 1, 'rotten': 2, 'unripe': 3}


In [ ]:
model.save('banana_classifier_model.h5')